In [2]:
from imutils import contours
from skimage import measure
import numpy as np
import imutils
import cv2
import pandas as pd
import glob
import swifter

import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

In [4]:
df = pd.read_csv('exceptions.csv')

In [5]:
def get_folder(path):
    return path.split('/')[-2]

def get_file(path):
    return path.split('/')[-1]

In [6]:
df['folder'] = df.path.swifter.progress_bar(True).apply(get_folder)

In [7]:
df['file'] = df.path.swifter.progress_bar(True).apply(get_file)

In [8]:
df

,path,raises_exception,folder,file
0,dataset/ocular/0/2342_left.jpg,NaN,0,2342_left.jpg
1,dataset/ocular/0/3407_right.jpg,NaN,0,3407_right.jpg
2,dataset/ocular/0/2343_left.jpg,NaN,0,2343_left.jpg
3,dataset/ocular/0/3294_right.jpg,NaN,0,3294_right.jpg
4,dataset/ocular/0/940_right.jpg,NaN,0,940_right.jpg
...,...,...,...,...
3419,dataset/ocular/1/1964_left.jpg,NaN,1,1964_left.jpg
3420,dataset/ocular/1/1265_right.jpg,NaN,1,1265_right.jpg
3421,dataset/ocular/1/1965_left.jpg,NaN,1,1965_left.jpg
3422,dataset/ocular/1/1503_right.jpg,NaN,1,1503_right.jpg


In [20]:
def get_disk(row):
  try:
    image = row.path
    folder = row.folder
    file = row.file
    image = cv2.imread(image)
    
    width = 512
    dim = (width, width)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    crop = 100
    image = image[100:width-100, 100:width-100]
    
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.bilateralFilter(gray,9,75,75)
    median=cv2.medianBlur(blur, 5)

    # threshold the image to reveal light regions in the
    # blurred image
    thresh = cv2.threshold(median, 155, 255, cv2.THRESH_BINARY)[1]
    # perform a series of erosions and dilations to remove
    # any small blobs of noise from the thresholded image
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=4)

    # perform a connected component analysis on the thresholded
    # image, then initialize a mask to store only the "large"
    # components
    labels = measure.label(thresh, connectivity=2, background=0)
    mask = np.zeros(thresh.shape, dtype="uint8")

    # loop over the unique components
    for label in np.unique(labels):
      # if this is the background label, ignore it
      if label == 0:
        continue

      # otherwise, construct the label mask and count the
      # number of pixels 
      labelMask = np.zeros(thresh.shape, dtype="uint8")
      labelMask[labels == label] = 255
      numPixels = cv2.countNonZero(labelMask)

      # if the number of pixels in the component is sufficiently
      # large, then add it to our mask of "large blobs"
      if numPixels >300:
        mask = cv2.add(mask, labelMask)

    # find the contours in the mask, then sort them from left to
    # right
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
      cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = contours.sort_contours(cnts)[0]
    # print (cnts)
    # loop over the contours
    for (i, c) in enumerate(cnts):
        ellipse = cv2.fitEllipse(c)
        (x, y, w, h) = cv2.boundingRect(c)
        break
      # draw the bright spot on the image
    color_coverted = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil_image=Image.fromarray(color_coverted)
    pil_image.save("disk_dataset_ocular/"+str(folder)+"/"+file)
  except:
    print (row.file)

In [21]:
df.head()

,path,raises_exception,folder,file
0,../dataset_generation/kaggle_dataset/0/189.jpg,True,0,189.jpg
1,../dataset_generation/kaggle_dataset/0/162.jpg,True,0,162.jpg
2,../dataset_generation/kaggle_dataset/0/176.jpg,True,0,176.jpg
3,../dataset_generation/kaggle_dataset/0/610.jpg,True,0,610.jpg
4,../dataset_generation/kaggle_dataset/0/360.jpg,True,0,360.jpg


In [22]:
df.swifter.progress_bar(True).apply(get_disk, axis=1)

(pid=11776) 233.jpg


0      None
1      None
2      None
3      None
4      None
       ... 
645    None
646    None
647    None
648    None
649    None
Length: 650, dtype: object